In [110]:
!pip install pyLDAvis 

In [112]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
import gensim
from gensim.models import CoherenceModel
from gensim import corpora
import pandas as pd
from pprint import pprint
import string
import os
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nhgirma/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [118]:
model_df = pd.read_csv('clean_processed_tweet_data.csv')
model_df = model_df.fillna("")
model_df.head()

,created_at,statuses_count,source,original_text,clean_text,sentiment,polarity,subjectivity,lang,favorite_count,retweet_count,original_author,screen_count,followers_count,friends_count,possibly_sensitive,hashtags,user_mentions,place,place_coord_boundaries
0,2022-08-07 22:31:20+00:00,8097,"<a href=""http://twitter.com/download/android"" ...",RT @i_ameztoy: Extra random image (I):\n\nLets...,\n\nLets focus in one very specific zone of th...,-1,0.000000e+00,0.08125,en,15760,2,i_ameztoy,232,20497,2621,,City,,,
1,2022-08-07 22:31:16+00:00,5831,"<a href=""http://twitter.com/download/android"" ...",RT @IndoPac_Info: #China's media explains the ...,#China's media explains the military reasons ...,0,-1.000000e-01,0.10000,en,6967,201,ZIisq,3,65,272,,China,,,
2,2022-08-07 22:31:07+00:00,1627,"<a href=""http://twitter.com/download/android"" ...","China even cut off communication, they don't a...","China even cut off communication, they don't a...",-1,0.000000e+00,0.00000,en,2166,0,Fin21Free,0,85,392,,XiJinping,,Netherlands,
3,2022-08-07 22:31:06+00:00,1627,"<a href=""http://twitter.com/download/android"" ...","Putin to #XiJinping : I told you my friend, Ta...","Putin to #XiJinping : I told you my friend, Ta...",1,1.000000e-01,0.35000,en,2166,0,Fin21Free,0,85,392,,XiJinping,,Netherlands,
4,2022-08-07 22:31:04+00:00,18958,"<a href=""http://twitter.com/download/iphone"" r...","RT @ChinaUncensored: I’m sorry, I thought Taiw...","I’m sorry, I thought Taiwan was an independen...",0,-6.938894e-18,0.55625,en,17247,381,VizziniDolores,0,910,2608,,,,"Ayent, Schweiz",


In [119]:
model_df.shape

(22000, 20)

In [120]:
sentiment_data = model_df.copy(deep=True)
sentiment_data.drop(sentiment_data[sentiment_tweet_data['sentiment'] == -1].index, inplace=True)
sentiment_data.reset_index(drop=True, inplace=True)
train_set = sentiment_tweet_data.iloc[:17600,]
test_set = sentiment_tweet_data.iloc[17601:,]

In [121]:
unigram_vectorizer = CountVectorizer(ngram_range=(1, 1))
unigram_vectorizer.fit(train_set['clean_text'].values)

CountVectorizer()

In [67]:
X_train_unigram = unigram_vectorizer.transform(train_set['clean_text'].values)

In [68]:
unigram_tf_idf_transformer = TfidfTransformer()
unigram_tf_idf_transformer.fit(X_train_unigram)

TfidfTransformer()

In [69]:
X_train_unigram_tf_idf = unigram_tf_idf_transformer.transform(X_train_unigram)

In [70]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2))
bigram_vectorizer.fit(train_set['clean_text'].values)

CountVectorizer(ngram_range=(1, 2))

In [71]:
X_train_bigram = bigram_vectorizer.transform(train_set['clean_text'].values)

In [72]:
bigram_tf_idf_transformer = TfidfTransformer()
bigram_tf_idf_transformer.fit(X_train_bigram)

TfidfTransformer()

In [47]:
X_train_bigram_tf_idf = bigram_tf_idf_transformer.transform(X_train_bigram)

In [73]:
def train_and_show_scores(X: csr_matrix, y: np.array, title: str) -> None:
    X_train, X_valid, y_train, y_valid = train_test_split(
        X, y,train_size=0.75, stratify=y
    )

    clf = SGDClassifier()
    clf.fit(X_train, y_train)
    train_score = clf.score(X_train, y_train)
    valid_score = clf.score(X_valid, y_valid)

    global_vars = globals()
    if(valid_score > global_vars['best_score']):
        global_vars['best_model'] = clf
        global_vars['best_model_name'] = title
        global_vars['best_score'] = valid_score

    print(f'{title}\nTrain score: {round(train_score, 2)} ; Validation score: {round(valid_score, 2)}\n')

In [74]:
y_train = train_set['sentiment'].values
y_train

array([-1,  0, -1, ..., -1, -1,  1])

In [55]:
best_model = ""
best_model_name = ""
best_score = 0

train_and_show_scores(X_train_unigram, y_train, 'Unigram Counts')
train_and_show_scores(X_train_unigram_tf_idf, y_train, 'Unigram Tf-Idf')
train_and_show_scores(X_train_bigram, y_train, 'Bigram Counts')
train_and_show_scores(X_train_bigram_tf_idf, y_train, 'Bigram Tf-Idf')

Unigram Counts
Train score: 1.0 ; Validation score: 0.94

Unigram Tf-Idf
Train score: 0.98 ; Validation score: 0.93

Bigram Counts
Train score: 1.0 ; Validation score: 0.93

Bigram Tf-Idf
Train score: 0.99 ; Validation score: 0.92



In [75]:
print(f'Best Model: {best_model_name} \nValidation Score: {round(best_score, 2)}')


Best Model: Unigram Counts 
Validation Score: 0.94


In [101]:
sgd = joblib.dump(best_model, './trained_model/newsentimentSGDmodel.jl')

In [117]:
from nltk.corpus import stopwords
stop_words= stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])


In [88]:
class PrepareData:
      def __init__(self,df):
        self.df=df
    
      def preprocess_data(self):
        tweets_df = self.df.loc[self.df['lang'] =="en"]

    
    #text Preprocessing
        model_df['clean_text']=model_df['clean_text'].astype(str)
        model_df['clean_text'] = model_df['clean_text'].apply(lambda x: x.lower())
        model_df['clean_text']= model_df['clean_text'].apply(lambda x: x.translate(str.maketrans(' ', ' ', string.punctuation)))

        #Converting tweets to list of words For feature engineering
        sentence_list = [tweet for tweet in model_df['clean_text']]
        word_list = [sent.split() for sent in sentence_list]
        # print(word_list)

        #Create dictionary which contains Id and word 
        word_to_id = corpora.Dictionary(word_list) #generate unique tokens
        #  we can see the word to unique integer mapping
        # print(word_to_id.token2id)
        # using bag of words(bow), we create a corpus that contains the word id and its frequency in each document.
        corpus_1= [word_to_id.doc2bow(tweet) for tweet in word_list]
        

        return word_list, word_to_id, corpus_1


In [89]:
PrepareData_obj=PrepareData(model_df)
word_list ,id2word,corpus=PrepareData_obj.preprocess_data()

In [113]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
model_df['clean_text'] = df['clean_text'].apply(lambda x: " ".join([stemmer.stem(word) for word in x.split()]))


In [114]:
model_df["clean_text"]

0        let focu in one veri specif zone of the wester...
1        china media explain the militari reason for ea...
2        china even cut off commun they dont anwer phon...
3        putin to xijinp i told you my friend taiwan wi...
4        i’m sorri i thought taiwan wa an independ coun...
                               ...                        
21995    a good infograph of china missil launch on tai...
21996    a good infograph of china missil launch on tai...
21997                       reuter thank pelosi smart move
21998    taiwan peopl desir for unif with china near re...
21999    taiwan peopl desir for unif with china near re...
Name: clean_text, Length: 22000, dtype: object

In [97]:
print(corpus[0:3])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1)], [(5, 2), (8, 1), (13, 5), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1)], [(13, 2), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 2), (51, 1), (52, 1), (53, 1)]]


In [94]:
id_words = [[(id2word[id], count) for id, count in line] for line in corpus]

In [96]:
print(id_words[0:3])

[[('city', 1), ('coast', 1), ('district', 1), ('focus', 1), ('gt', 1), ('in', 1), ('lets', 1), ('longjing', 1), ('of', 1), ('one', 1), ('specific', 1), ('taichung', 1), ('ta…', 1), ('the', 1), ('very', 1), ('western', 1), ('zone', 1)], [('in', 2), ('of', 1), ('the', 5), ('area', 1), ('chinas', 1), ('drills', 1), ('each', 1), ('explains', 1), ('for', 1), ('labels', 1), ('media', 1), ('military', 1), ('pi…', 1), ('read', 1), ('reasons', 1), ('strait', 1), ('taiwan', 1)], [('the', 2), ('anwer', 1), ('ask', 1), ('but', 1), ('change', 1), ('china', 1), ('clown', 1), ('communication', 1), ('cut', 1), ('dont', 1), ('enters', 1), ('even', 1), ('from', 1), ('here', 1), ('mind', 1), ('off', 1), ('phonecalls', 1), ('putins', 1), ('stage', 1), ('they', 1), ('to', 2), ('us', 1), ('xijinping', 1), ('zelenskyyua', 1)]]


In [98]:
type(id_words)

list

In [102]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [103]:
pprint(lda_model.print_topics())

[(0,
  '0.044*"the" + 0.033*"on" + 0.027*"taiwan" + 0.022*"chinese" + '
  '0.020*"island" + 0.019*"in" + 0.019*"to" + 0.016*"us" + 0.016*"as" + '
  '0.014*"of"'),
 (1,
  '0.037*"taiwan" + 0.021*"china" + 0.019*"taiwans" + 0.019*"is" + '
  '0.018*"that" + 0.017*"you" + 0.015*"video" + 0.013*"of" + 0.013*"a" + '
  '0.013*"it"'),
 (2,
  '0.047*"of" + 0.033*"missile" + 0.024*"part" + 0.019*"delegation" + '
  '0.019*"we" + 0.018*"her" + 0.017*"amp" + 0.014*"deputy" + 0.012*"to…" + '
  '0.012*"some"'),
 (3,
  '0.038*"to" + 0.031*"in" + 0.019*"taiwan" + 0.018*"was" + 0.017*"on" + '
  '0.016*"visit" + 0.016*"the" + 0.014*"by" + 0.013*"i" + 0.011*"is"'),
 (4,
  '0.068*"the" + 0.047*"taiwan" + 0.030*"to" + 0.030*"of" + 0.024*"and" + '
  '0.024*"a" + 0.021*"china" + 0.021*"for" + 0.019*"is" + 0.014*"in"')]


In [104]:
pprint(lda_model.show_topics(formatted=False))

[(0,
  [('the', 0.04395358),
   ('on', 0.033439633),
   ('taiwan', 0.026925098),
   ('chinese', 0.02212068),
   ('island', 0.0204998),
   ('in', 0.019174475),
   ('to', 0.018654335),
   ('us', 0.016076317),
   ('as', 0.015548982),
   ('of', 0.014487119)]),
 (1,
  [('taiwan', 0.03713237),
   ('china', 0.020807313),
   ('taiwans', 0.019410735),
   ('is', 0.018574348),
   ('that', 0.017751308),
   ('you', 0.016795183),
   ('video', 0.015092552),
   ('of', 0.013485775),
   ('a', 0.013134908),
   ('it', 0.012776444)]),
 (2,
  [('of', 0.046632145),
   ('missile', 0.03342878),
   ('part', 0.024009362),
   ('delegation', 0.01924059),
   ('we', 0.019214101),
   ('her', 0.01789514),
   ('amp', 0.017246703),
   ('deputy', 0.013842348),
   ('to…', 0.012484315),
   ('some', 0.012330312)]),
 (3,
  [('to', 0.037837338),
   ('in', 0.030890705),
   ('taiwan', 0.018933749),
   ('was', 0.018106118),
   ('on', 0.017319217),
   ('visit', 0.016418425),
   ('the', 0.016400944),
   ('by', 0.0143851815),
   ('

In [105]:
# Compute Perplexity

#It's a measure of how good the model is. The lower the better. Perplexity is a negative value
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  
doc_lda = lda_model[corpus]


# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=word_list, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\n Ldamodel Coherence Score/Accuracy on Tweets: ', coherence_lda)


Perplexity:  -7.699028621882877

 Ldamodel Coherence Score/Accuracy on Tweets:  0.2714030745837813


In [107]:
import pyLDAvis.gensim_models as gensimvis
import pickle 
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()

LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word)
LDAvis_prepared

/home/nhgirma/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/nhgirma/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/nhgirma/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/nhgirma/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/nhgirma/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: Deprecatio

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.045042 -0.134748       1        1  40.358285
3     -0.221424  0.044663       2        1  18.579610
0     -0.115318 -0.057490       3        1  17.014744
1      0.265190 -0.134701       4        1  16.557708
2      0.116594  0.282276       5        1   7.489652, topic_info=        Term          Freq         Total Category  logprob  loglift
8         of   8397.000000   8397.000000  Default  30.0000  30.0000
174  missile   1849.000000   1849.000000  Default  29.0000  29.0000
22       for   3506.000000   3506.000000  Default  28.0000  28.0000
13       the  16234.000000  16234.000000  Default  27.0000  27.0000
5         in   6121.000000   6121.000000  Default  26.0000  26.0000
..       ...           ...           ...      ...      ...      ...
50        to    377.247460  10099.956276   Topic5  -4.4140  -0.6957
113      and    280.198728   5504.347412   Topic5  -4.7114  -0.3861
13       the    318.648484  16234.316072   Topic5  -4.5828  -1.3391
117       is    221.192379   6169.912299   Topic5  -4.9479  -0.7368
30    taiwan    232.895519  14032.029536   Topic5  -4.8963  -1.5069

[285 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
274       3  0.998924     3
167       3  0.998100   4th
54        1  0.659792     a
54        2  0.104330     a
54        3  0.032040     a
...     ...       ...   ...
76        2  0.082042   you
76        4  0.912714   you
76        5  0.004733   you
541       4  0.998292  your
1230      5  0.995041   佩洛西

[309 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 1, 2, 3])